In [ ]:
import os
import re

#tunning data filter
index = 0
lines = []
filterlines = [] 

regex = r'\[(.*)\]'

file_object = open('/Users/sunxiaoyong/Downloads/docker-gdal2/data1/part2', 'r')
# lines = file_object.readlines()

for line in file_object.readlines():
    index += 1
    lines.append(line)
    if(index == 10):
        break
file_object.close()        
    
for i in lines:
    match_obj = re.finditer(regex, i)
    
# loop the filtered data and delete the useless information
    for m in match_obj:
        match1 = m.group(0)

# append elements to list
        filterlines.append(match1)
print(filterlines)  

#radar-processing data filter
index2 = 0
lines2 = []
filterlines2 = [] 

regex2 = r'([0-9]*\.?[0-9]*)\[(.*)\]'

file_object2 = open('/Users/sunxiaoyong/Downloads/docker-gdal2/data2/part1', 'r')
# lines = file_object.readlines()

for l in file_object2.readlines():
    index2 += 1
    lines2.append(l)
    if(index2 == 32):
        break
file_object.close()        
    
for i in lines2:
    match_obj2 = re.finditer(regex2, i)
    
# loop the filtered data and delete the useless information
    for m in match_obj2:
        match2 = m.group(0)

# append elements to list
        filterlines2.append(match2)
print(filterlines2)  

#make the provenance
from prov.model import ProvDocument

d3 = ProvDocument()
d3.set_default_namespace('http://setting.org/')
d3.add_namespace('rp', 'http://radar-processing.org/')
d3.add_namespace('setting','http://setting.org/')

tunning = d3.activity('rp:tunning',("20/06/2018"))
scanning = d3.activity('rp:scanning')
filtering = d3.activity('rp:filtering')

data = d3.entity('rp:data')
hdf_a = d3.entity('rp:hdf_a')
hdf_b = d3.entity('rp:hdf_b')

d3.wasGeneratedBy(hdf_a,filtering)
d3.wasGeneratedBy(hdf_b,filtering)

read=d3.activity('rp:read')

d3.used(read,hdf_a)
d3.used(read,hdf_b)

d3.wasInformedBy(scanning,tunning)

d3.wasGeneratedBy(data,scanning)

d3.used(filtering,data)

tunning_setting = d3.entity('tunning_attributes',{'setting:azimuth_velocity':filterlines[0],
                                                  'setting:cellsvalue':filterlines[1],
                                                  'setting:range_step_value':filterlines[2],
                                                  'setting:range_zero_delay_value':filterlines[3],
                                                  'setting:pulse_width_value':filterlines[4],
                                                  'setting:jitter':filterlines[5],
                                                  'setting:tune_indicator_threshold':filterlines[6],
                                                  'setting:azimuth_position_to':filterlines[7],
                                                  'setting:level_one_elevation':filterlines[8],
                                                  'setting:level_three_elevation':filterlines[9],
                                                 })
filter_setting = d3.entity('filter_attributes',{'setting:NOISE_MEAN_LEVEL':filterlines2[0],
                                                'setting:25NOISE_MAX_LEVEL':filterlines2[1],
                                                'setting:25NOISE_MIN_LEVEL':filterlines2[2],
                                                'setting:NOISE_PEAK_LEVEL_SIGNAL':filterlines2[3],
                                                'setting:NOISE_STANDARDD_EVIATION_LEVEL':filterlines2[4],
                                                'setting:NOISE_RMS_LEVEL':filterlines2[5],
                                                'setting:RANGE_A':filterlines2[6],
                                                'setting:NUMBER_OF_EMPTY_CELLS_A':filterlines2[7],
                                                'setting:NUMBER_OF_GROUND_CLUTTER_CELLS_A':filterlines2[8],
                                                'setting:NUMBER_OF_MIXED_SIGNAL_CELLS_A':filterlines2[9],
                                                'setting:NUMBER_OF_WEATHER_SIGNAL_CELLS_A':filterlines2[10],
                                                'setting:NUMBER_OF_EMPTY_CELLS_B':filterlines2[11],
                                                'setting:NUMBER_OF_GROUND_CLUTTER_CELLS_B':filterlines2[12],
                                                'setting:NUMBER_OF_MIXED_SIGNAL_CELLS_B':filterlines2[13],
                                                'setting:NUMBER_OF_WEATHER_SIGNAL_CELLS_B':filterlines2[14],
                                                'setting:NUMBER_OF_EMPTY_CELLS_C':filterlines2[15],
                                                'setting:NUMBER_OF_GROUND_CLUTTER_CELLS_C':filterlines2[16],
                                                'setting:NUMBER_OF_MIXED_SIGNAL_CELLS_C':filterlines2[17],
                                                'setting:NUMBER_OF_WEATHER_SIGNAL_CELLS_C':filterlines2[18],
                                                'setting:ELAPSED_TIME':filterlines2[19],
                                                'setting:RANGE_B':filterlines2[20],
                                                'setting:NUMBER_OF_EMPTY_CELLS_D':filterlines2[21],
                                                'setting:MIXED_SIGNAL_CELLS_D':filterlines2[22],
                                                'setting:MIXED_SIGNAL_CELLS':filterlines2[23],
                                                'setting:WEATHER_SIGNAL_CELLS':filterlines2[24],
                                                'setting:THEORIC_DECISION_THRESHOLD':filterlines2[25],
                                                'setting:THEORIC_MIN_LOW_THRESHOLD':filterlines2[26],
                                                'setting:THEORIC_MAX_LOW_THRESHOLD':filterlines2[27],
                                                'setting:MEAN_THEORIC_DECISION_THRESHOLD':filterlines2[28],
                                                'setting:MEAN_THEORIC_MIN_LOW_THRESHOLD':filterlines2[29],
                                                'setting:MEAN_THEORIC_MAX_LOW_THRESHOLD':filterlines2[30],
                                                'setting:ELAPSED_TIME':filterlines2[31],
                                               })

d3.used(tunning,tunning_setting)
d3.used(filtering,filter_setting)
d3.used(tunning,data)

print(d3.get_provn())
from prov.dot import prov_to_dot
dot = prov_to_dot(d3)
dot.write_png('article-prov.png')
# from IPython.display import Image
# Image('article-prov.png')